In [1]:
import re
import datetime
import csv

import numpy as np
import pandas as pd

from elasticsearch import Elasticsearch, helpers

pd.options.display.max_colwidth = 100
pd.set_option('display.max_rows', 300)

In [2]:
#es = Elasticsearch([{'host': 'hcc-metrics.unl.edu', 'port': 9200}])
username = 'cms'
password = 'B3A96-D0g_PkkPz5'
es = Elasticsearch([{'host': 'es-cms.cern.ch', 'port': 9203, 'http_auth': username+':'+password}], use_ssl=True,verify_certs=True,ca_certs="ca-bundle.trust.crt")

In [3]:
indices = es.cat.indices(index="cms-*", h="index", request_timeout=600).split('\n')
indices = sorted(indices)
indices = [x for x in indices if x != '']

In [4]:
def time_filter(indices, days=0, until=0):
    if days == 0:
        return ["cms-*"]
    today = datetime.date.today()
    filtered = []
    datefmt = '%Y-%m-%d'
    for i in indices:
        day = re.sub(r'cms-', '', i).rstrip()
        day = datetime.datetime.strptime(day, datefmt).date()
        diff = today - day
        if until <= diff.days < days + until:
            filtered.append(i.rstrip())
    return filtered

In [5]:
# Change second argument to use a different time range for ES queries
ind = time_filter(indices, 90, 0)
ind = ','.join(ind)
print ind

cms-2017-03-11,cms-2017-03-12,cms-2017-03-13,cms-2017-03-14,cms-2017-03-15,cms-2017-03-16,cms-2017-03-17,cms-2017-03-18,cms-2017-03-19,cms-2017-03-20,cms-2017-03-21,cms-2017-03-22,cms-2017-03-23,cms-2017-03-24,cms-2017-03-25,cms-2017-03-26,cms-2017-03-27,cms-2017-03-28,cms-2017-03-29,cms-2017-03-30,cms-2017-03-31,cms-2017-04-01,cms-2017-04-02,cms-2017-04-03,cms-2017-04-04,cms-2017-04-05,cms-2017-04-06,cms-2017-04-07,cms-2017-04-08,cms-2017-04-09,cms-2017-04-10,cms-2017-04-11,cms-2017-04-12,cms-2017-04-13,cms-2017-04-14,cms-2017-04-15,cms-2017-04-16,cms-2017-04-17,cms-2017-04-18,cms-2017-04-19,cms-2017-04-20,cms-2017-04-21,cms-2017-04-22,cms-2017-04-23,cms-2017-04-24,cms-2017-04-25,cms-2017-04-26,cms-2017-04-27,cms-2017-04-28,cms-2017-04-29,cms-2017-04-30,cms-2017-05-01,cms-2017-05-02,cms-2017-05-03,cms-2017-05-04,cms-2017-05-05,cms-2017-05-06,cms-2017-05-07,cms-2017-05-08,cms-2017-05-09,cms-2017-05-10,cms-2017-05-11,cms-2017-05-12,cms-2017-05-13,cms-2017-05-14,cms-2017-05-15,cms-2017-0

In [6]:
s = {
    "size": 10000,
    "_source": ["CMS_JobType", "WMAgent_RequestName", "WMAgent_SubTaskName", "WMAgent_TaskType"],
    "query": {
        "bool": {
            "filter": {
                "exists": {"field": "WMAgent_RequestName"}
            }
        }
    },
    "aggs": {
        "req": {
            "terms": {
                "field": "WMAgent_RequestName",
                "size": 0,
            },
            "aggs": {
                "task": {
                    "terms": {
                        "field": "WMAgent_SubTaskName",
                        "size": 0,
                    },
                    "aggs": {
                        "ttype": {
                            "terms": {
                                "field": "WMAgent_TaskType",
                                "size": 0
                            },
                            "aggs": {
                                "jtype": {
                                    "terms": {
                                        "field": "CMS_JobType",
                                        "size": 0
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }
}
res = es.search(index=ind, body=s, request_timeout=1200)

In [7]:
with open('wmagent.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['req', 'task', 'ttype', 'jtype', 'njobs'])
    for b1 in res['aggregations']['req']['buckets']:
        req = b1['key']
        for b2 in b1['task']['buckets']:
            task = b2['key']
            for b3 in b2['ttype']['buckets']:
                ttype = b3['key']
                for b4 in b3['jtype']['buckets']:
                    jtype = b4['key']
                    njobs = b4['doc_count']
                writer.writerow([req, task, ttype, jtype, njobs])

In [8]:
df = pd.read_csv('wmagent.csv')

In [9]:
def tasktype(row):
    req = row['req']
    task = row['task']
    ttype = row['ttype']
    jtype = row['jtype'].lower()
    ttype = ttype.lower()

    if "cleanupunmerged" in ttype:
        tt = "Cleanup"
    elif "dataprocessing" in ttype:
        tt = "DataProcessing"
    elif "merge" in ttype:
        tt = "Merge"
    elif "logcollect" in ttype:
        tt = 'LogCollect'
    elif "express" in ttype:
        tt = "Express"
    elif "reco" in ttype and "promptreco" in task:
        tt = "PromptReco"
    elif "repack" in ttype:
        tt = "Repack"
    elif "alcaskim" in ttype:
        tt = "AlcaSkim"
    elif ("miniaod" in req.lower()) and (ttype == "steponeproc"):
        tt = "MINIAOD"
    elif ttype == "steponeproc":
        tt = 'DIGIRECO'
    elif ttype == "steptwoproc":
        tt = 'RECO'
    elif ttype == "stepthreeproc":
        tt = 'MINIAODSIM'
    elif "miniaodv" in ttype:
        tt = 'MINIAOD'
    elif ("gs-" in req.lower()) and ttype.endswith("_0"):
        tt = 'GENSIM'
    elif ttype.endswith("_0"):
        tt = 'DIGI'
    elif "digifull" in ttype:
        tt = 'DIGIFULL'
    elif "digihi" in ttype:
        tt = 'DIGIHI'
    elif ttype.endswith("_1"):
        tt = 'RECO'
    elif "recoprod" in ttype:
        tt = 'RECOPROD'
    elif "recohi" in ttype:
        tt = 'RECOHI'
    elif "recofull" in ttype:
        tt = 'RECOFULL'
    elif ttype == "montecarlofromgen":
        tt = 'GENSIM'
    elif "gensim" in ttype:
        tt = 'GENSIM'
    elif (ttype == 'production') and ("gs-" in req.lower()):
        tt = 'GENSIM'
    elif (ttype == 'production') and ("lhe-" in req.lower()):
        tt = 'LHE'
    elif (ttype == 'production') and ("premix-" in req.lower()):
        tt = 'PREMIX'
    elif "premix" in ttype:
        tt = 'PREMIX'
    elif "validation" in req.lower():
        tt = "Validation"
    elif "relval" in req.lower():
        tt = "RelVal"
    elif "dqm" in ttype:
        tt = "DQM"
    elif "skim" in ttype:
        tt = "Skim"
    elif ttype == "production":
        tt = "GENSIM"
    else:
        tt = ttype
    return tt

In [10]:
def tasktype2(row):
    req = row['req'].lower()
    task = row['task'].lower()
    ttype = row['ttype'].lower()
    jtype = row['jtype'].lower()

    if 'cleanupunmerged' in ttype:
        tt = 'Cleanup'
    elif 'logcollect' in ttype:
        tt = 'LogCollect'
    elif ttype == 'repack':
        tt = 'Repack'
    elif ttype.startswith('skim'):
        tt = 'Skim'
    elif 'harvest' in ttype:
        tt = 'Harvesting'
    elif 'merge' in ttype:
        tt = 'Merge'
    elif ttype.startswith('express'):
        tt = 'Express'
    elif ttype == 'alcaskim':
        tt = 'Alcaskim'
    elif 'gensim' in ttype:
        tt = 'GENSIM'
    elif ttype == 'montecarlofromgen':
        tt = 'GENSIM'
    elif ttype == 'gen':
        tt = 'GEN'
    elif ttype == 'reco':
        tt = 'RECO'
    elif ttype == 'dataprocessing':
        tt = 'RECO'
    elif ttype.startswith("reco"):
        tt = 'RECO'
    elif ttype.endswith("_0"):
        tt = 'DIGI'
    elif ttype.endswith("_1"):
        tt = 'DIGIRECO'
    elif ttype.startswith("digi"):
        tt = 'DIGI'
    elif ttype == 'steponeproc':
        tt = 'DIGI'
    elif ttype == 'steptwoproc':
        tt = 'DIGIRECO'
    elif ttype == 'stepthreeproc':
        tt = 'DIGIRECOMINIAOD'
    elif 'miniaod' in ttype:
        tt = 'MINIAOD'
    elif ttype == 'production':
        tt = 'GENSIM'
    elif 'hlt' in ttype:
        tt = 'RECO'
    elif "validation" in task:
        tt = "Validation"
    elif "relval" in task:
        tt = "RelVal"
    elif "fastsim" in task:
        tt = "FastSim"
    elif "premix" in task:
        tt = "Premix"
    elif "minbias" in task:
        tt = "MinBias"
    else:
        tt = "Other"

    return tt
    

In [11]:
df['tasktype'] = df.apply(lambda row: tasktype2(row), axis=1)

In [12]:
grouped = df.groupby(['ttype'], as_index=False)
a = grouped.agg({'njobs': 'sum'})

In [13]:
a.sort_values('njobs', ascending=False)

,ttype,njobs
43373,production,1878301
5162,dataprocessing,1651226
45260,steponeproc,1402615
45083,smp-runiisummer16dr80premix-00199_0,484106
13879,express,423550
45290,steptwoproc,416835
45085,smp-runiisummer16dr80premix-00199_1,389749
44316,repack,385965
5248,dataprocessingmergeaodoutput,377227
3887,bph-phaseifall16gs-00022_0,369194


In [14]:
grouped = df.groupby(['tasktype'], as_index=False)
a = grouped.agg({'njobs': 'sum'})

In [15]:
a.sort_values('njobs', ascending=False)

,tasktype,njobs
2,DIGI,18461796
3,DIGIRECO,7046319
11,Merge,2699794
7,GENSIM,2244108
15,RECO,2233079
9,LogCollect,1288761
1,Cleanup,1179435
5,Express,468478
17,Repack,385965
0,Alcaskim,144584


Unfortunately several task types escape classification, namely a large number of 'production' jobs, 'steptwoproc', 'stepthreeproc', 'digiup15', etc.

It is my guess that 'steptwoproc' is a DIGI (or better a ReDigi), looking at the WmAgent code.

Similarly, 'dataprocessing' should be RECO.

In [16]:
print df[df.ttype == 'steponeproc'][:40]

                                                                             req  \
280            pdmvserv_smp-phaseiifall16dr82-00026_00044_v0__170316_145557_1828   
907            pdmvserv_smp-phaseiifall16dr82-00027_00045_v0__170318_174420_7100   
1041          pdmvserv_hin-ppb502winter16dr-00002_00001_v0_mb_170410_150359_4959   
1129          pdmvserv_hin-ppb502winter16dr-00001_00001_v0_mb_170410_150500_5622   
1139          pdmvserv_hin-ppb502winter16dr-00003_00001_v0_mb_170410_150409_5898   
1149          pdmvserv_hin-ppb502winter16dr-00004_00001_v0_mb_170410_150424_8830   
1575            pdmvserv_egm-phaseispring17dr-00019_00004_v0__170424_164431_3526   
1577            pdmvserv_egm-phaseispring17dr-00024_00005_v0__170424_165757_5177   
1615            pdmvserv_egm-phaseispring17dr-00023_00005_v0__170424_165749_2406   
1632            pdmvserv_egm-phaseispring17dr-00020_00004_v0__170424_164438_1771   
1649             pdmvserv_egm-phaseispring17dr-00018_00006_v1__170425_161919